In [1]:
!pip install -q transformers

In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline

In [9]:
# Import business data
# Rename to avoid two columns called 'stars'
yelp_business = pd.read_feather('../../FilteredData/business_philly.feather')
yelp_business = yelp_business.rename(columns={"stars": "business_avgStars"})
yelp_business.head(3)

,index,_id,business_id,name,address,city,state,postal_code,latitude,longitude,business_avgStars,review_count,is_open,attributes,categories,hours
0,31,631ea3b2a5cde8cc0d6eec47,-0M0b-XhtFagyLmsBtOe8w,Paris Wine Bar,2303 Fairmount Ave,Philadelphia,PA,19130,39.967439,-75.175452,3.5,18,0,"{'Alcohol': ""u'full_bar'"", 'OutdoorSeating': '...","Bars, Nightlife, Restaurants, French, Wine Bars","{'Thursday': '17:0-0:0', 'Friday': '17:0-0:0',..."
1,32,631ea3b0a5cde8cc0d6dfa60,-0PN_KFPtbnLQZEeb23XiA,Mr Wong's Chinese Restaurant,1849 Wolf St,Philadelphia,PA,19145,39.923048,-75.178078,3.5,9,0,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Chinese","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
2,33,631ea3aea5cde8cc0d6d5a50,-0TffRSXXIlBYVbb5AwfTg,IndeBlue Modern Indian Food & Spirits,205 South 13th St,Philadelphia,PA,19107,39.948508,-75.161969,4.5,1097,1,"{'RestaurantsReservations': 'True', 'NoiseLeve...","Cocktail Bars, Food Delivery Services, Nightli...","{'Monday': '0:0-0:0', 'Tuesday': '16:0-22:0', ..."


In [6]:
# Import reviews data
yelp_reviews = pd.read_feather('../../FilteredData/review_philly.feather')
yelp_reviews.head(3)

,_id,review_id,user_id,business_id,stars,useful,funny,cool,text,date,compliment_count
0,631e9f7fedf65856ab0dfe3b,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,NaN
1,631e9f7fedf65856ab0dfe3e,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0.0,0.0,0.0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14,NaN
2,631e9f7fedf65856ab0dfe42,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,NaN


In [10]:
# Look at one particular review
yelp_reviews['text'][0]

"I am a long term frequent customer of this establishment. I just went in to order take out (3 apps) and was told they're too busy to do it. Really? The place is maybe half full at best. Does your dick reach your ass? Yes? Go fuck yourself! I'm a frequent customer AND great tipper. Glad that Kanella just opened. NEVER going back to dmitris!"

In [19]:
# Combine both business & reviews data
#yelp_reviews_business = pd.concat((yelp_reviews, yelp_business), axis=1)
#yelp_reviews_business = yelp_reviews_business[['review_id', 'user_id', 'business_id','stars','text',
#                                                                 'categories', 'business_avgStars']]
# For now drop lines with blanks for error handling, can figure out how to deal with this later
yelp_reviews_business_filtered = yelp_reviews[['user_id','review_id','business_id','stars','text','date']]
yelp_reviews_business_filtered.head(3)

,user_id,review_id,business_id,stars,text,date
0,eUta8W_HdHMXPzLBBZhL1A,JrIxlS1TzJ-iCu79ul40cQ,04UD14gamNjLY0IDYVhHJg,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31
1,smOvOajNG0lS4Pq7d8g4JQ,8JFGBuHMoiNDyfcxuWNtrA,RZtGWDLCAtuipwaZ-UfjmQ,4.0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14
2,_7bHUi9Uuf5__HHc_Q8guQ,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03


In [21]:
yelp_reviews_business_filtered.shape

(855080, 6)

In [22]:
yelp_reviews_business_filtered["Sentiment_stars"] = yelp_reviews_business_filtered["stars"].apply(lambda score: "positive" if score >= 3 else "negative")
yelp_reviews_business_filtered["Sentiment_stars_v2"] = yelp_reviews_business_filtered["stars"].apply(lambda score: "positive" if score > 3 else "negative")
yelp_reviews_business_filtered = yelp_reviews_business_filtered.reset_index()
yelp_reviews_business_filtered.head(3)

<ipython-input-22-0de2bdcf01c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_reviews_business_filtered["Sentiment_stars"] = yelp_reviews_business_filtered["stars"].apply(lambda score: "positive" if score >= 3 else "negative")
<ipython-input-22-0de2bdcf01c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_reviews_business_filtered["Sentiment_stars_v2"] = yelp_reviews_business_filtered["stars"].apply(lambda score: "positive" if score > 3 else "negative")


,index,user_id,review_id,business_id,stars,text,date,Sentiment_stars,Sentiment_stars_v2
0,0,eUta8W_HdHMXPzLBBZhL1A,JrIxlS1TzJ-iCu79ul40cQ,04UD14gamNjLY0IDYVhHJg,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,negative,negative
1,1,smOvOajNG0lS4Pq7d8g4JQ,8JFGBuHMoiNDyfcxuWNtrA,RZtGWDLCAtuipwaZ-UfjmQ,4.0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14,positive,positive
2,2,_7bHUi9Uuf5__HHc_Q8guQ,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,positive,positive


In [23]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [24]:
preds = pd.DataFrame()

for i in range(yelp_reviews_business_filtered.shape[0]):
    data = yelp_reviews_business_filtered['text'][i][:512]
    df_pred = pd.DataFrame(sentiment_pipeline(data))
    preds = pd.concat((preds,df_pred))

preds = preds.reset_index()
yelp_sentiment_distilbert = pd.concat((yelp_reviews_business_filtered,preds), axis = 1)

In [28]:
yelp_sentiment_distilbert.shape

(855080, 12)

In [29]:
yelp_sentiment_distilbert.to_csv("sentiment_results_distilbert.csv")

In [30]:
sentiment_sameCount = yelp_sentiment_distilbert[yelp_sentiment_distilbert['label'].str.lower()==yelp_sentiment_distilbert['Sentiment_stars'].str.lower()].shape[0]

In [31]:
sentiment_diffCount = yelp_sentiment_distilbert[yelp_sentiment_distilbert['label'].str.lower()!=yelp_sentiment_distilbert['Sentiment_stars'].str.lower()].shape[0]

In [32]:
print("Same sentiment label: ", sentiment_sameCount)
print("Different sentiment label: ", sentiment_diffCount)

Same sentiment label:  733591
Different sentiment label:  121489


In [95]:
df_diffSentiment = yelp_sentiment_distilbert[yelp_sentiment_distilbert['label'].str.lower()!=yelp_sentiment_distilbert['Sentiment_stars'].str.lower()]

In [97]:
df_diffSentiment['text'][5]

"The bun makes the Sonoran Dog. It's like a snuggie for the pup. A first, it seems ridiculous and almost like it's going to be too much, exactly like everyone's favorite blanket with sleeves. Too much softness, too much smush, too indulgent.  Wrong. It's warm, soft, chewy, fragrant, and it succeeds where other famed Sonoran Dogs fail. \n\nThe hot dog itself is flavorful, but I would prefer that it or the bacon have a little more bite or snap to better hold their own against the dominant mustard and onions. \n\nI'm with the masses on the carne asada caramelo. Excellent tortilla, salty, melty cheese, and great carne. \n\nSuper cheap and you can drive through."

In [100]:
df_diffSentiment[['stars', 'label', 'score']]

,stars,label,score
5,4,NEGATIVE,0.986714
10,1,POSITIVE,0.962190
13,5,NEGATIVE,0.984514
31,1,POSITIVE,0.999411
39,5,NEGATIVE,0.966219
...,...,...,...
33367,3,NEGATIVE,0.980247
33380,3,NEGATIVE,0.997115
33390,3,NEGATIVE,0.986983
33398,4,NEGATIVE,0.695079


In [101]:
df_diffSentiment['text'][10]

'If you want to pay for everything a la carte this is the place for you.  \nFood wasn\'t terrible not impressive.\nThey brought a basket of chips and some tomato sauce which I asked politely for something spicier and some pico de gallo.  She brought them happily to me and the salsa was much better.  When asked what we would like to drink I asked for a coke and she brought out a bottle which I stated I wanted the fountain drink.  She said "oh that\'s only Pepsi".  Never mentioned that they only had bottle drinks for coke.  \nWe ordered our food which was reasonably priced, asked for sour cream and also to put cheese on the taco.  She let us know cheese was extra.  \nIt was $2.50 extra for another basket of chips.  \nWhen I received the bill we paid more for the condiments then the actual food.  Side of sour cream 2.00, pico de gallo 2.50, salsa 2.00, chips 2.50, cheese 1.00 and the bottled coke that we didn\'t want 5.00.  \nJust a suggestion...when you order anything make sure to ask if

In [3]:
df_sentiment = pd.read_csv(r'sentiment_results_distilbert.csv')

In [5]:
df_2 = df_sentiment[['user_id','label']]

In [6]:
df_4 = df_2.groupby(['user_id','label']).size().sort_values(ascending=False).reset_index(name='count') 

In [7]:
df_4

,user_id,label,count
0,ET8n-r7glWYqZhuR6GcdNw,POSITIVE,879
1,_BcWyKQL16ndpBdggh2kNA,POSITIVE,841
2,bJ5FtCtZX3ZZacz2_2PJjA,POSITIVE,749
3,vFd8aBLg1kFcd0kCkoi-xw,POSITIVE,538
4,8EMU7d4pCkdqUnvlIW40CA,POSITIVE,449
...,...,...,...
302227,P0Z-QXDbhB59alC3ztNEsw,POSITIVE,1
302228,P0Z2wH0FFrIQ1ktTtRLx4w,POSITIVE,1
302229,P0a4jmRppSqqiCcbrJK8MQ,POSITIVE,1
302230,P0a7W89WqdL4s9eFksKl5A,POSITIVE,1


In [20]:
df_5 = df_4.groupby(['user_id']).sum(['count']).reset_index() 

In [21]:
df_5

,user_id,count
0,---r61b7EpVPkb4UVme5tA,1
1,--0kuuLmuYBe3Rmu0Iycww,2
2,--13zE3NaRvLSrmfTVnFJA,1
3,--2tyArRmSoyKx5r-FVG0A,6
4,--2vR0DIsmQ6WfcSzKWigw,5
...,...,...
245406,zzu2hGJ6O9mP5yg6fjtvzg,1
245407,zzvCl_egPyWpxO7EvWc2IA,3
245408,zzwYLnmIvj8C7wJmRjtkRA,2
245409,zzx7J3zheFF3zf5YYfDAMg,1


In [11]:
df_6 = df_4[df_4['label']=="POSITIVE"]
df_6

,user_id,label,count
0,ET8n-r7glWYqZhuR6GcdNw,POSITIVE,879
1,_BcWyKQL16ndpBdggh2kNA,POSITIVE,841
2,bJ5FtCtZX3ZZacz2_2PJjA,POSITIVE,749
3,vFd8aBLg1kFcd0kCkoi-xw,POSITIVE,538
4,8EMU7d4pCkdqUnvlIW40CA,POSITIVE,449
...,...,...,...
302226,P0YnaoMwa-tPjqGrNkbTcQ,POSITIVE,1
302227,P0Z-QXDbhB59alC3ztNEsw,POSITIVE,1
302228,P0Z2wH0FFrIQ1ktTtRLx4w,POSITIVE,1
302229,P0a4jmRppSqqiCcbrJK8MQ,POSITIVE,1


In [15]:
df_6 = df_6.rename(columns={'count': 'posCount'})

In [23]:
df_7 = df_6.merge(df_5, on="user_id", how="left")

In [24]:
df_7

,user_id,label,posCount,count
0,ET8n-r7glWYqZhuR6GcdNw,POSITIVE,879,1179
1,_BcWyKQL16ndpBdggh2kNA,POSITIVE,841,1217
2,bJ5FtCtZX3ZZacz2_2PJjA,POSITIVE,749,1011
3,vFd8aBLg1kFcd0kCkoi-xw,POSITIVE,538,666
4,8EMU7d4pCkdqUnvlIW40CA,POSITIVE,449,518
...,...,...,...,...
190582,P0YnaoMwa-tPjqGrNkbTcQ,POSITIVE,1,1
190583,P0Z-QXDbhB59alC3ztNEsw,POSITIVE,1,1
190584,P0Z2wH0FFrIQ1ktTtRLx4w,POSITIVE,1,1
190585,P0a4jmRppSqqiCcbrJK8MQ,POSITIVE,1,1


In [25]:
df_7['overall_percentage'] = df_7['posCount']/df_7['count']

In [32]:
df_8 = df_7[['user_id','overall_percentage']]

In [29]:
user_data = pd.read_feather(r'../../FilteredData/user_philly.feather')

In [33]:
user_data.merge(df_8, on="user_id", how="left")

,index,_id,user_id,name,review_count,yelping_since,useful,funny,cool,elite,...,compliment_photos,gender_score,philly_reviews,philly_share_of_reviews,bucketed_philly_share_of_reviews,bucketed_gender_scores,average_stars_given,bucketed_average_stars_given,bucketed_philly_reviews,overall_percentage
0,4,631ea1f7abab926ea88770d7,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",...,1946,0.995754,13,0.003000,less_than_or_equal_to_25_percent,more_than_80_percent,4.0,more_than_3_up_to_4,more_than_5_up_to_25,0.923077
1,6,631ea1f7abab926ea88770d9,AUi8MPWJ0mLkMfwbui27lg,John,109,2010-01-07 18:32:04,154,20,23,,...,0,0.995785,2,0.018349,less_than_or_equal_to_25_percent,more_than_80_percent,2.0,more_than_1_up_to_2,more_than_1_up_to_5,NaN
2,12,631ea1f7abab926ea88770df,1McG5Rn_UDkmlkZOrsdptg,Teresa,7,2009-05-26 16:11:11,18,3,13,,...,0,0.002806,5,0.714286,more_than_50_up_to_75_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,more_than_1_up_to_5,1.000000
3,21,631ea1f7abab926ea88770e8,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,"2006,2007,2008,2009,2010,2011,2012,2013,2014",...,323,0.003043,4,0.003276,less_than_or_equal_to_25_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,more_than_1_up_to_5,0.750000
4,22,631ea1f7abab926ea88770e9,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,...,180,0.986824,3,0.005128,less_than_or_equal_to_25_percent,more_than_80_percent,5.0,more_than_4,more_than_1_up_to_5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245406,1987805,631ea2f2abab926ea8a5c5b0,Tr2yeddopDzMhSb7jbdNeA,Heather,1,2017-07-25 19:31:30,0,0,0,,...,0,0.002816,1,1.000000,more_than_75_percent,less_than_or_equal_to_20_percent,5.0,more_than_4,just_1,1.000000
245407,1987831,631ea2f2abab926ea8a5c5ca,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,2017-11-24 03:17:11,1,0,0,,...,0,0.000000,1,0.166667,less_than_or_equal_to_25_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,just_1,1.000000
245408,1987832,631ea2f2abab926ea8a5c5cb,iZ0puydkFQlaSQAXbGtv1g,Dr. Sylvia,1,2014-07-08 19:54:19,2,0,0,,...,0,0.500000,1,1.000000,more_than_75_percent,more_than_40_up_to_60_percent,5.0,more_than_4,just_1,1.000000
245409,1987859,631ea2f2abab926ea8a5c5e6,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,2015-03-23 17:29:22,0,0,0,,...,0,0.011541,1,0.500000,more_than_25_up_to_50_percent,less_than_or_equal_to_20_percent,5.0,more_than_4,just_1,1.000000


In [44]:
user_data = pd.read_feather(r'../../FilteredData/user_philly.feather').rename(columns={'overall_percentage': 'positive_reviews_over_total_reviews'})

In [46]:
user_data.to_feather(r'../../FilteredData/user_philly.feather')

In [47]:
pd.read_feather(r'../../FilteredData/user_philly.feather')

,index,_id,user_id,name,review_count,yelping_since,useful,funny,cool,elite,...,compliment_photos,gender_score,philly_reviews,philly_share_of_reviews,bucketed_philly_share_of_reviews,bucketed_gender_scores,average_stars_given,bucketed_average_stars_given,bucketed_philly_reviews,positive_reviews_over_total_reviews
0,4,631ea1f7abab926ea88770d7,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...",...,1946,0.995754,13,0.003000,less_than_or_equal_to_25_percent,more_than_80_percent,4.0,more_than_3_up_to_4,more_than_5_up_to_25,0.923077
1,6,631ea1f7abab926ea88770d9,AUi8MPWJ0mLkMfwbui27lg,John,109,2010-01-07 18:32:04,154,20,23,,...,0,0.995785,2,0.018349,less_than_or_equal_to_25_percent,more_than_80_percent,2.0,more_than_1_up_to_2,more_than_1_up_to_5,0.000000
2,12,631ea1f7abab926ea88770df,1McG5Rn_UDkmlkZOrsdptg,Teresa,7,2009-05-26 16:11:11,18,3,13,,...,0,0.002806,5,0.714286,more_than_50_up_to_75_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,more_than_1_up_to_5,1.000000
3,21,631ea1f7abab926ea88770e8,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,"2006,2007,2008,2009,2010,2011,2012,2013,2014",...,323,0.003043,4,0.003276,less_than_or_equal_to_25_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,more_than_1_up_to_5,0.750000
4,22,631ea1f7abab926ea88770e9,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,...,180,0.986824,3,0.005128,less_than_or_equal_to_25_percent,more_than_80_percent,5.0,more_than_4,more_than_1_up_to_5,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245406,1987805,631ea2f2abab926ea8a5c5b0,Tr2yeddopDzMhSb7jbdNeA,Heather,1,2017-07-25 19:31:30,0,0,0,,...,0,0.002816,1,1.000000,more_than_75_percent,less_than_or_equal_to_20_percent,5.0,more_than_4,just_1,1.000000
245407,1987831,631ea2f2abab926ea8a5c5ca,XTWARBzLbiJKQ1JeJB9f3g,Sakina,6,2017-11-24 03:17:11,1,0,0,,...,0,0.000000,1,0.166667,less_than_or_equal_to_25_percent,less_than_or_equal_to_20_percent,4.0,more_than_3_up_to_4,just_1,1.000000
245408,1987832,631ea2f2abab926ea8a5c5cb,iZ0puydkFQlaSQAXbGtv1g,Dr. Sylvia,1,2014-07-08 19:54:19,2,0,0,,...,0,0.500000,1,1.000000,more_than_75_percent,more_than_40_up_to_60_percent,5.0,more_than_4,just_1,1.000000
245409,1987859,631ea2f2abab926ea8a5c5e6,K-BNvWUCWEGUuBirCU4mmg,Jodi,2,2015-03-23 17:29:22,0,0,0,,...,0,0.011541,1,0.500000,more_than_25_up_to_50_percent,less_than_or_equal_to_20_percent,5.0,more_than_4,just_1,1.000000
